In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os
import numpy as np
import time
import random
import warnings
warnings.filterwarnings("ignore")

# 读取数据集

In [ ]:
data_org_dir = 'data/data_tencent/'
data_prep_dir = 'data/preprocess/'

In [2]:
df_data = pd.read_csv('data/preprocess/df_train_test_merge.csv')